<a href="https://colab.research.google.com/github/Muthumaha-S/Alzheimer_Prediction/blob/main/Alzhiemer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
import os

drive.mount('/content/drive')

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/"))

IMPORT LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

 Load and Preprocess Dataset

In [ ]:
import os

DATA_DIR = "/content/drive/MyDrive/archive/alziemer_dataset/"  # Check if this exists

# Check if the dataset path exists
print("Dataset Directory Exists:", os.path.exists(DATA_DIR))

# If exists, list the contents
if os.path.exists(DATA_DIR):
    print("Dataset Structure:", os.listdir(DATA_DIR))
else:
    print("Error: Dataset path is incorrect!")

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/archive/alziemer_dataset/"
IMG_SIZE = 128  # Resize all images to 128x128
BATCH_SIZE = 32  # Batch size for training

In [ ]:
print("Dataset Exists:", os.path.exists(DATASET_PATH))
print("Dataset Structure:", os.listdir(DATASET_PATH) if os.path.exists(DATASET_PATH) else "Path not found")

# Check individual category folders
for folder in ["train", "test", "val"]:
    path = os.path.join(DATASET_PATH, folder)
    print(f"{folder.capitalize()} Categories:", os.listdir(path) if os.path.exists(path) else "Not Found")

Load Data Function

In [ ]:
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Define categories
CATEGORIES = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Function to load images from dataset
def load_data(directory):
    images, labels = [], []

    for label_idx, label in enumerate(CATEGORIES):
        class_path = os.path.join(directory, label)

        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)

                # Ensure the file is an image
                if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    continue

                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Skipping unreadable image: {img_path}")
                    continue  # Skip unreadable images

                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
                images.append(img)
                labels.append(label_idx)

    # Convert to numpy arrays
    images = np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # Add channel dimension
    labels = to_categorical(labels, num_classes=len(CATEGORIES))

    return images.astype('float32'), labels

# Load training, validation, and test data
X_train, y_train = load_data(os.path.join(DATASET_PATH, "train"))
X_val, y_val = load_data(os.path.join(DATASET_PATH, "val"))
X_test, y_test = load_data(os.path.join(DATASET_PATH, "test"))

# Print dataset statistics
print(f"✅ Train set: {X_train.shape}, Labels: {y_train.shape}")
print(f"✅ Validation set: {X_val.shape}, Labels: {y_val.shape}")
print(f"✅ Test set: {X_test.shape}, Labels: {y_test.shape}")

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

def build_unet_model():
    inputs = Input((IMG_SIZE, IMG_SIZE, 1))  # Grayscale MRI images , 1 channel because these are grayscale MRI scans.

    # Encoder (Downsampling)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    # Decoder (Upsampling)
    u1 = UpSampling2D((2, 2))(c3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)

    u2 = UpSampling2D((2, 2))(c4)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)  # Binary segmentation mask

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

unet_model = build_unet_model()
unet_model.summary()

In [ ]:
import matplotlib.pyplot as plt

def plot_segmentation(index):
    # Get original image
    original = X_test[index].squeeze()

    # Predict segmentation mask
    predicted_mask = unet_model.predict(X_test[index].reshape(1, IMG_SIZE, IMG_SIZE, 1)).squeeze()

    # Threshold the predicted mask to make it binary
    binary_mask = (predicted_mask > 0.5).astype('float32')

    # Plot original and segmented mask side by side
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original MRI")

    plt.subplot(1, 3, 2)
    plt.imshow(binary_mask, cmap='gray')
    plt.title("Segmented Brain Mask")

    # Overlay segmentation on original for better visualization
    plt.subplot(1, 3, 3)
    plt.imshow(original, cmap='gray')
    plt.imshow(binary_mask, cmap='jet', alpha=0.5)  # Overlay with transparency
    plt.title("Overlay (Segmentation + MRI)")

    plt.show()

# Example usage
plot_segmentation(10)

#  Define a function to convert grayscale images to RGB format
def convert_to_rgb(dataset):
    return np.repeat(dataset, 3, axis=-1)  # Convert (128, 128, 1) → (128, 128, 3) # Function takes a dataset (4D array: samples, height, width, channels)

X_train_rgb = convert_to_rgb(X_train)
X_val_rgb = convert_to_rgb(X_val)
X_test_rgb = convert_to_rgb(X_test)

print(f"✅ RGB Training Data Shape: {X_train_rgb.shape}")

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Define ResNet-101 Model
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze all ResNet layers Freezing prevents overfitting
for layer in base_model.layers:
    layer.trainable = False

# Add Custom Layers for Classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # ✅ Fix applied here
x = Dense(512, activation='relu')(x) #Adds a fully connected (dense) layer with 512 neurons.
x = Dropout(0.5)(x) #Adds Dropout regularization with a dropout rate of 50%.   Prevents overfitting by forcing the network to learn redundant, robust features.
output_layer = Dense(len(CATEGORIES), activation='softmax')(x)

# Create Final Model
resnet_model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print Model Summary
resnet_model.summary()

In [ ]:
history_resnet = resnet_model.fit(
    X_train_rgb, y_train,
    validation_data=(X_val_rgb, y_val),
    epochs=20,
    batch_size=32
)

VISUALIZATION REPORT

In [ ]:
y_pred = np.argmax(resnet_model.predict(X_test_rgb), axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report:\n", classification_report(y_true, y_pred, target_names=CATEGORIES))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CATEGORIES, yticklabels=CATEGORIES)
plt.title('Confusion Matrix - ResNet101')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


In [ ]:
#Bar Plot of Predictions per Category

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Bar plot of predictions per category
plt.figure(figsize=(8, 5))
sns.countplot(x=[CATEGORIES[i] for i in y_pred], order=CATEGORIES, palette="Set2")
plt.title("Predicted Distribution of Classes")
plt.xlabel("Dementia Category")
plt.ylabel("Number of Predictions")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()


In [ ]:
#Training & Validation Accuracy and Loss Curves

# Accuracy plot
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history_resnet.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history_resnet.history['val_accuracy'], label='Val Accuracy', marker='o')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history_resnet.history['loss'], label='Train Loss', marker='o')
plt.plot(history_resnet.history['val_loss'], label='Val Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
def predict_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    img = np.expand_dims(img, axis=-1)  # Convert to (128, 128, 1)
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    img_rgb = convert_to_rgb(img)  # Convert to (128, 128, 3) for ResNet

    pred = np.argmax(resnet_model.predict(img_rgb), axis=1)[0]
    print(f"Prediction: {CATEGORIES[pred]}")

predict_image("/content/drive/MyDrive/ig.jpg")


In [ ]:
resnet_model.save('/content/drive/MyDrive/alzheimers_prediction/alzheimer_prediction_model.h5')
